## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Hermanus,ZA,-34.4187,19.2345,55.90,75,0,10.56,clear sky
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,23.00,100,90,11.50,mist
2,2,Vaini,TO,-21.2000,-175.2000,74.21,66,54,17.83,broken clouds
3,3,Hobart,AU,-42.8794,147.3294,52.27,60,75,1.01,light rain
4,4,Kavieng,PG,-2.5744,150.7967,86.07,66,58,14.99,broken clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 75


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Hermanus,ZA,-34.4187,19.2345,55.90,75,0,10.56,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,74.21,66,54,17.83,broken clouds
3,3,Hobart,AU,-42.8794,147.3294,52.27,60,75,1.01,light rain
5,5,Port Elizabeth,ZA,-33.9180,25.5701,47.10,74,1,6.26,clear sky
9,9,Puerto Ayora,EC,-0.7393,-90.3518,69.89,84,80,8.61,broken clouds
11,11,Cidreira,BR,-30.1811,-50.2056,60.87,77,39,7.63,scattered clouds
12,12,Dawei,MM,14.0833,98.2000,73.56,89,96,4.36,overcast clouds
16,16,San Cristobal,VE,7.7669,-72.2250,68.20,87,58,1.90,broken clouds
21,21,Busselton,AU,-33.6500,115.3333,60.04,95,100,20.38,overcast clouds
22,22,East London,ZA,-33.0153,27.9116,57.72,70,58,6.11,broken clouds


In [22]:
# 4a. Determine if there are any empty rows.
city_data_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current description    0
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=city_data_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current description    0
dtype: int64

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Hermanus,ZA,55.90,clear sky,-34.4187,19.2345,
2,Vaini,TO,74.21,broken clouds,-21.2000,-175.2000,
3,Hobart,AU,52.27,light rain,-42.8794,147.3294,
5,Port Elizabeth,ZA,47.10,clear sky,-33.9180,25.5701,
9,Puerto Ayora,EC,69.89,broken clouds,-0.7393,-90.3518,
11,Cidreira,BR,60.87,scattered clouds,-30.1811,-50.2056,
12,Dawei,MM,73.56,overcast clouds,14.0833,98.2000,
16,San Cristobal,VE,68.20,broken clouds,7.7669,-72.2250,
21,Busselton,AU,60.04,overcast clouds,-33.6500,115.3333,
22,East London,ZA,57.72,broken clouds,-33.0153,27.9116,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Hermanus,ZA,55.90,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
2,Vaini,TO,74.21,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
3,Hobart,AU,52.27,light rain,-42.8794,147.3294,Mantra on Collins Hobart
5,Port Elizabeth,ZA,47.10,clear sky,-33.9180,25.5701,39 On Nile Guest House
9,Puerto Ayora,EC,69.89,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Cidreira,BR,60.87,scattered clouds,-30.1811,-50.2056,Hotel Castelo
12,Dawei,MM,73.56,overcast clouds,14.0833,98.2000,Hotel Dawei
16,San Cristobal,VE,68.20,broken clouds,7.7669,-72.2250,Posada Villaven C.A.
21,Busselton,AU,60.04,overcast clouds,-33.6500,115.3333,Observatory Guest House
22,East London,ZA,57.72,broken clouds,-33.0153,27.9116,Tu Casa


In [31]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name']!= ' ']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Hermanus,ZA,55.90,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
2,Vaini,TO,74.21,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
3,Hobart,AU,52.27,light rain,-42.8794,147.3294,Mantra on Collins Hobart
5,Port Elizabeth,ZA,47.10,clear sky,-33.9180,25.5701,39 On Nile Guest House
9,Puerto Ayora,EC,69.89,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Cidreira,BR,60.87,scattered clouds,-30.1811,-50.2056,Hotel Castelo
12,Dawei,MM,73.56,overcast clouds,14.0833,98.2000,Hotel Dawei
16,San Cristobal,VE,68.20,broken clouds,7.7669,-72.2250,Posada Villaven C.A.
21,Busselton,AU,60.04,overcast clouds,-33.6500,115.3333,Observatory Guest House
22,East London,ZA,57.72,broken clouds,-33.0153,27.9116,Tu Casa


In [36]:
# 8a. Create the output File (CSV)
output_data_file= 'WeatherPy_vacation.csv'
# 8b. Export the hotel_df into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Save Figure
plt.savefig("WeatherPy_vacation_map.png")
plt.show()